# Imports

In [6]:
import os
import pickle as pkl
from itertools import tee

import numpy as np
import open3d as o3d

import torch
from torch import nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.utils import train_test_split_edges
from torch_geometric.nn import GCNConv, SAGEConv, GATConv, GAE, LayerNorm, Linear
from torch_geometric.nn import global_mean_pool, global_max_pool

# Paths

In [2]:
# registeration_path = "../../local_data/registered_only/male/1000180/"
registeration_path = "../../local_data/registered_only/male/1000180"
pt_data = "../../local_data/organ_meshes/1000071/liver_mesh.pt"

# Data

In [3]:
registered_mesh = []
organ = "liver_mesh.ply"
mesh = o3d.io.read_triangle_mesh(os.path.join(registeration_path, organ))

vertices_data = np.asarray(mesh.vertices)
triangles = np.asarray(mesh.triangles)
vertices = torch.from_numpy(vertices_data).double()
edges = []
for triangle in triangles:
    edges.append([triangle[0], triangle[1]])
    edges.append([triangle[0], triangle[2]])
    edges.append([triangle[1], triangle[2]])
edges_torch = torch.from_numpy(np.unique(np.array(edges), axis=0).reshape(2,-1)).long()

registered_mesh.append((vertices.type(torch.float32), edges_torch))
data = Data(x=registered_mesh[0][0], edge_index=registered_mesh[0][1], num_nodes= len(registered_mesh[0][0]))
data = train_test_split_edges(data)
data

c:\Users\yussu\anaconda3\envs\local_digital_twin_env\lib\site-packages\torch_geometric\deprecation.py:22: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


AttributeError: module 'torch' has no attribute 'sparse_csr'

# Functions

In [6]:
def get_gnn_layers(num_conv_layers: int, hidden_channels, num_inp_features:int, 
                 gnn_layer, activation=nn.ReLU, normalization=None, dropout = None):
    """Creates GNN layers"""
    layers = nn.ModuleList()

    for i, j in enumerate(range(1,num_conv_layers,1)):
        if i == 0:
            layers.append(gnn_layer(num_inp_features, hidden_channels[i]))
            layers.append(activation())
            if normalization is not None:
                layers.append(normalization(hidden_channels[i]))
        else:
            layers.append(gnn_layer(hidden_channels[i-1], hidden_channels[i]))
            layers.append(activation())
            if normalization is not None:
                layers.append(normalization(hidden_channels[i]))

    return nn.ModuleList(layers)

def get_mlp_layers(channels: list, activation, output_activation=nn.Identity):
    """Define basic multilayered perceptron network."""
    layers = []
    *intermediate_layer_definitions, final_layer_definition = pairwise(channels)

    for in_ch, out_ch in intermediate_layer_definitions:
        intermediate_layer = nn.Linear(in_ch, out_ch)
        layers += [intermediate_layer, activation()]

    layers += [nn.Linear(*final_layer_definition), output_activation()]
    #print('Output activation ',output_activation)
    return nn.Sequential(*layers)

def pairwise(iterable):
    """Iterate over all pairs of consecutive items in a list.
    Notes
    -----
        [s0, s1, s2, s3, ...] -> (s0,s1), (s1,s2), (s2, s3), ...
    """
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

def train(model, optimizer, x, train_pos_edge_index):
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    loss.backward()
    optimizer.step()
    return float(loss)


def test(model, x, train_pos_edge_index, pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

# Model

In [7]:
class GNN(torch.nn.Module):
    def __init__(self, in_features, hidden_channels, activation, normalization, num_conv_layers=4, layer='gcn',
                 use_input_encoder=True, encoder_features=128, apply_batch_norm=True,
                 apply_dropout_every=True, dropout = 0):
        super(GNN, self).__init__()
        torch.manual_seed(42)
        
        self.fc = torch.nn.ModuleList()
        self.layer_type = layer
        self.use_input_encoder = use_input_encoder
        self.apply_batch_norm = apply_batch_norm
        self.dropout = dropout
        self.normalization_bool = normalization
        self.activation = activation
        self.apply_dropout_every = apply_dropout_every

        if self.normalization_bool:
            self.normalization = LayerNorm
        else:
            self.normalization = None

        if self.use_input_encoder :
            self.input_encoder = get_mlp_layers(
                channels=[in_features, encoder_features],
                activation=nn.ELU,
            )
            in_features = encoder_features

        if layer == 'gcn':
            self.layers = get_gnn_layers(num_conv_layers, hidden_channels, num_inp_features=in_features,
                                        gnn_layer=GCNConv,activation=activation,normalization=self.normalization )
        elif layer == 'sageconv':
            self.layers = get_gnn_layers(num_conv_layers, hidden_channels,in_features,
                                        gnn_layer=SAGEConv,activation=activation,normalization=self.normalization )
        elif layer == 'gat':
            self.layers = get_gnn_layers(num_conv_layers, hidden_channels,in_features,
                                        gnn_layer=GATConv,activation=activation,normalization=self.normalization )        

    def forward(self, x, edge_index):

        if self.use_input_encoder:
            x = self.input_encoder(x)

        if self.normalization is None:
            for i, layer in enumerate(self.layers):
                # Each GCN consists 2 modules GCN -> Activation 
                # GCN send edge index
                if i% 2 == 0:
                    x = layer(x, edge_index)
                else:
                    x = layer(x)

                if self.apply_dropout_every:
                    x = F.dropout(x, p=self.dropout, training=self.training)
        else:
            for i, layer in enumerate(self.layers):
                # Each GCN consists 3 modules GCN -> Activation ->  Normalization 
                # GCN send edge index
                if i% 3 == 0:
                    x = layer(x, edge_index)
                else:
                    x = layer(x)

                if self.apply_dropout_every:
                    x = F.dropout(x, p=self.dropout, training=self.training)        
        
        return x

# Model params

In [4]:
activation = getattr(nn, "ReLU")
model_params = dict(
        use_input_encoder = False,
        in_features= 3, 
        encoder_features = 128,
        hidden_channels= [512, 256, 128, 64],
        activation=activation,
        normalization = True,
        layer = "gcn",
        num_conv_layers = 4,
        dropout = 0.5)
epochs = 100

In [9]:
# model
model = GAE(GNN(**model_params))

# move to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

AttributeError: 'GlobalStorage' object has no attribute 'train_pos_edge_index'

In [ ]:
for epoch in range(1, epochs + 1):
    loss = train(model, optimizer, x, train_pos_edge_index)

    test_pos_edge_index = data.test_pos_edge_index.to(device)
    test_neg_edge_index = data.test_neg_edge_index.to(device)

    auc, ap = test(model, x, train_pos_edge_index, test_pos_edge_index, test_neg_edge_index)
    if(epoch % 10 == 0):
        print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

In [ ]:
Z = model.encode(x, train_pos_edge_index)
print(f'latent space: {Z}')

# Pooling

In [37]:
from torch_geometric.nn import TopKPooling, SAGPooling
import torch
import torch.nn as nn
x = torch.zeros(4,5, dtype=int)
print(x)
pool_layer = TopKPooling(in_channels=5, ratio=64)
x = pool_layer(x,x)
print(x)

tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]])
(tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]], grad_fn=<MulBackward0>), tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]), None, tensor([0, 0, 0, 0]), tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.], grad_fn=<IndexBackward0>))
